In [ ]:
import os
import pandas as pd
csv_dir = <file directory>




In [ ]:
csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
df_columns = {}
for file in csv_files:
    file_path = os.path.join(csv_dir, file)
    df = pd.read_csv(file_path)
    df_columns[file] = list(df.columns)
for file, columns in df_columns.items():
    print(f"{file}: {columns}")

REVENUE_HFS.csv: ['Unnamed: 0', 'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA', 'Reference area', 'FREQ', 'Frequency of observation', 'MEASURE', 'Measure', 'UNIT_MEASURE', 'Unit of measure', 'FINANCING_SCHEME', 'Financing scheme', 'FINANCING_SCHEME_REV', 'Revenues of financing schemes', 'FUNCTION', 'Health function', 'MODE_PROVISION', 'Mode of provision', 'PROVIDER', 'Health care provider', 'FACTOR_PROVISION', 'Factor of provision', 'ASSET_TYPE', 'Asset type', 'PRICE_BASE', 'Price base', 'TIME_PERIOD', 'OBS_VALUE', 'UNIT_MULT', 'Unit multiplier', 'DECIMALS', 'Decimals']
HealthGFCF.csv: ['Unnamed: 0', 'STRUCTURE', 'STRUCTURE_ID', 'STRUCTURE_NAME', 'ACTION', 'REF_AREA', 'Reference area', 'FREQ', 'Frequency of observation', 'MEASURE', 'Measure', 'UNIT_MEASURE', 'Unit of measure', 'FINANCING_SCHEME', 'Financing scheme', 'FINANCING_SCHEME_REV', 'Revenues of financing schemes', 'FUNCTION', 'Health function', 'MODE_PROVISION', 'Mode of provision', 'PROVIDER', 'Health care

In [ ]:
import pandas as pd
import os
import numpy as np
from datetime import datetime


file_paths = {
    'REVENUE_HFS': os.path.join(csv_dir, 'REVENUE_HFS.csv'),
    'HealthGFCF': os.path.join(csv_dir, 'HealthGFCF.csv'),
    'HP_cost': os.path.join(csv_dir, 'HP_cost.csv'),
    'HC_Coverage': os.path.join(csv_dir, 'HC_Coverage.csv'),
    'HEF_Main': os.path.join(csv_dir, 'HEF_Main.csv')
}
dfs = {}
for name, path in file_paths.items():
    try:
        if os.path.exists(path):
            dfs[name] = pd.read_csv(path)
            print(f"Successfully loaded {name}")
        else:
            print(f"Warning: File not found - {path}")
    except Exception as e:
        print(f"Error loading {name}: {str(e)}")

if not dfs:
    raise ValueError("No data files could be loaded. Please check file paths.")
def create_healthcare_star_schema(dfs):
    star_schema = {}
    
    # --------- DIMENSION TABLES ---------
    
    # 1. Geography Dimension (Dim_Geography)
    geography_frames = []
    for name, df in dfs.items():
        if 'REF_AREA' in df.columns and 'Reference area' in df.columns:
            geography_frames.append(df[['REF_AREA', 'Reference area']].copy())
    
    if geography_frames:
        geography_dim = pd.concat(geography_frames).drop_duplicates().reset_index(drop=True)
        # Add region and healthcare system type (placeholder values since they're not in source)
        geography_dim['Region'] = geography_dim['Reference area'].apply(
            lambda x: 'Europe' if x in ['Germany', 'France', 'United Kingdom', 'Italy', 'Spain'] 
            else ('North America' if x in ['United States', 'Canada'] else 'Other'))
        geography_dim['Healthcare_System_Type'] = geography_dim['Reference area'].apply(
            lambda x: 'Universal' if x != 'United States' else 'Mixed')
        geography_dim.columns = ['GEOGRAPHY_KEY', 'Country_Name', 'Region', 'Healthcare_System_Type']
        star_schema['Dim_Geography'] = geography_dim
    
    # 2. Time Dimension (Dim_Time)
    time_frames = []
    for name, df in dfs.items():
        if 'TIME_PERIOD' in df.columns:
            time_frames.append(df[['TIME_PERIOD']].copy())
    
    if time_frames:
        time_dim = pd.concat(time_frames).drop_duplicates().reset_index(drop=True)
        time_dim.columns = ['TIME_KEY']
        # Add additional time attributes
        time_dim['Year'] = time_dim['TIME_KEY'].astype(int)
        time_dim['Quarter'] = 'Q4'  # Default to Q4 since most health data is annual
        time_dim['Month'] = 12      # Default to December
        time_dim['FREQ'] = 'A'      # Annual frequency
        star_schema['Dim_Time'] = time_dim
    
    # 3. Insurance Type Dimension (Dim_InsuranceType)
    insurance_frames = []
    for name, df in dfs.items():
        if 'INSURANCE_TYPE' in df.columns and 'Insurance type' in df.columns:
            insurance_frames.append(df[['INSURANCE_TYPE', 'Insurance type']].copy())
    
    if insurance_frames:
        try:
            insurance_dim = pd.concat(insurance_frames).drop_duplicates().reset_index(drop=True)
            # Add coverage level and deductible (mock data)
            insurance_dim['Coverage_Level'] = np.random.choice(['Basic', 'Standard', 'Premium'], size=len(insurance_dim))
            insurance_dim['Deductible_Avg'] = np.random.randint(100, 5000, size=len(insurance_dim))
            insurance_dim['Copay_Percent'] = np.random.randint(10, 30, size=len(insurance_dim))
            insurance_dim.columns = ['INSURANCE_TYPE_KEY', 'INSURANCE_TYPE', 'Coverage_Level', 'Deductible_Avg', 'Copay_Percent']
            star_schema['Dim_InsuranceType'] = insurance_dim
        except Exception as e:
            print(f"Error creating insurance dimension: {e}")
            # Create mock insurance dimension if needed
            insurance_dim = pd.DataFrame({
                'INSURANCE_TYPE_KEY': ['PHI', 'SHI', 'OOP'],
                'INSURANCE_TYPE': ['Private Health Insurance', 'Social Health Insurance', 'Out-of-Pocket'],
                'Coverage_Level': ['Premium', 'Standard', 'Basic'],
                'Deductible_Avg': [2000, 500, 0],
                'Copay_Percent': [20, 15, 100]
            })
            star_schema['Dim_InsuranceType'] = insurance_dim
    
    # 4. Demographics Dimension (Dim_Demographics)
    # Since demographics aren't in source data, create mock dimension
    demographics_dim = pd.DataFrame({
        'DEMOGRAPHICS_KEY': range(1, 6),
        'Age_Group': ['18-24', '25-34', '35-44', '45-54', '55+'],
        'Income_Level': ['Low', 'Medium-Low', 'Medium', 'Medium-High', 'High'],
        'Employment_Status': ['Student', 'Employed', 'Employed', 'Employed', 'Retired']
    })
    star_schema['Dim_Demographics'] = demographics_dim
    
    # 5. Provider Dimension (Dim_Provider)
    provider_frames = []
    for name, df in dfs.items():
        if 'PROVIDER' in df.columns and 'Health care provider' in df.columns:
            provider_frames.append(df[['PROVIDER', 'Health care provider']].copy())
    
    if provider_frames:
        provider_dim = pd.concat(provider_frames).drop_duplicates().reset_index(drop=True)
        # Add provider type and ownership (mock data)
        provider_dim['Provider_Type'] = np.random.choice(['Hospital', 'Clinic', 'Pharmacy', 'Other'], size=len(provider_dim))
        provider_dim['Ownership'] = np.random.choice(['Public', 'Private', 'Non-Profit'], size=len(provider_dim))
        provider_dim.columns = ['PROVIDER_KEY', 'PROVIDER', 'Provider_Type', 'Ownership']
        star_schema['Dim_Provider'] = provider_dim
    
    # 6. Financing Scheme Dimension (Dim_FinancingScheme)
    financing_frames = []
    for name, df in dfs.items():
        if 'FINANCING_SCHEME' in df.columns and 'Financing scheme' in df.columns:
            financing_frames.append(df[['FINANCING_SCHEME', 'Financing scheme']].copy())
    
    if financing_frames:
        financing_dim = pd.concat(financing_frames).drop_duplicates().reset_index(drop=True)
        # Add revenue source (mock data)
        financing_dim['Revenue_Source'] = np.random.choice(['Tax', 'Premium', 'Direct Payment', 'Mixed'], size=len(financing_dim))
        financing_dim.columns = ['FINANCING_SCHEME_KEY', 'FINANCING_SCHEME', 'Revenue_Source']
        star_schema['Dim_FinancingScheme'] = financing_dim
    
    # --------- FACT TABLES ---------
    
    # 1. Insurance Coverage Outcomes Fact Table
    coverage_data = []
    for name in ['HC_Coverage', 'HEF_Main']:
        if name in dfs:
            df = dfs[name].copy()
            if all(col in df.columns for col in ['REF_AREA', 'TIME_PERIOD', 'INSURANCE_TYPE', 'OBS_VALUE']):
                coverage_data.append(df)
    
    if coverage_data:
        fact_coverage = pd.concat(coverage_data).reset_index(drop=True)
        # Add enrollment counts and satisfaction (mock data)
        fact_coverage['Insurance_Enrollment_Count'] = np.random.randint(1000, 1000000, size=len(fact_coverage))
        fact_coverage['Patient_Satisfaction_Score'] = np.random.uniform(3.0, 4.9, size=len(fact_coverage)).round(1)
        
        # Create keys for fact table
        fact_coverage['Coverage_ID'] = fact_coverage.index + 1
        fact_coverage['TIME_KEY'] = fact_coverage['TIME_PERIOD']
        fact_coverage['GEOGRAPHY_KEY'] = fact_coverage['REF_AREA']
        fact_coverage['INSURANCE_TYPE_KEY'] = fact_coverage['INSURANCE_TYPE']
        fact_coverage['DEMOGRAPHICS_KEY'] = np.random.randint(1, 6, size=len(fact_coverage))  # Random assignment
        
        # Select columns for fact table
        fact_coverage = fact_coverage[['Coverage_ID', 'TIME_KEY', 'GEOGRAPHY_KEY', 'INSURANCE_TYPE_KEY', 
                                       'DEMOGRAPHICS_KEY', 'OBS_VALUE', 'Insurance_Enrollment_Count', 
                                       'Patient_Satisfaction_Score']]
        star_schema['Fact_Insurance_Coverage_Outcomes'] = fact_coverage
    
    # 2. Healthcare Financing Fact Table
    financing_data = []
    for name in ['REVENUE_HFS', 'HealthGFCF', 'HP_cost']:
        if name in dfs:
            df = dfs[name].copy()
            if all(col in df.columns for col in ['REF_AREA', 'TIME_PERIOD', 'FINANCING_SCHEME', 'OBS_VALUE']):
                financing_data.append(df)
    
    if financing_data:
        fact_financing = pd.concat(financing_data).reset_index(drop=True)
        # Add per capita spending and %GDP (mock data)
        fact_financing['Per_Capita_Spending'] = np.random.randint(200, 12000, size=len(fact_financing))
        fact_financing['Percent_GDP'] = np.random.uniform(4.0, 18.0, size=len(fact_financing)).round(1)
        
        # Create keys for fact table
        fact_financing['Financing_ID'] = fact_financing.index + 1
        fact_financing['TIME_KEY'] = fact_financing['TIME_PERIOD']
        fact_financing['GEOGRAPHY_KEY'] = fact_financing['REF_AREA']
        fact_financing['FINANCING_SCHEME_KEY'] = fact_financing['FINANCING_SCHEME']
        fact_financing['PROVIDER_KEY'] = fact_financing['PROVIDER'] if 'PROVIDER' in fact_financing.columns else None
        
        # Select columns for fact table
        selected_columns = ['Financing_ID', 'TIME_KEY', 'GEOGRAPHY_KEY', 'FINANCING_SCHEME_KEY']
        if 'PROVIDER_KEY' in fact_financing.columns:
            selected_columns.append('PROVIDER_KEY')
        selected_columns.extend(['OBS_VALUE', 'Per_Capita_Spending', 'Percent_GDP'])
        
        fact_financing = fact_financing[selected_columns]
        star_schema['Fact_Healthcare_Financing'] = fact_financing
    
    return star_schema

# Create the star schema
try:
    star_schema = create_healthcare_star_schema(dfs)
    
    # Print sample of each table
    for table_name, df in star_schema.items():
        print(f"\n{table_name} (Shape: {df.shape}):")
        print(df.head(3))
    
    # Save tables to CSV (optional)
    output_dir = '/Users/vidushi/Documents/Datasets/DW/star_schema'
    os.makedirs(output_dir, exist_ok=True)
    
    for table_name, df in star_schema.items():
        output_path = os.path.join(output_dir, f"{table_name}.csv")
        df.to_csv(output_path, index=False)
        print(f"Saved {table_name} to {output_path}")
        
except Exception as e:
    print(f"Error creating star schema: {str(e)}")


Successfully loaded REVENUE_HFS
Successfully loaded HealthGFCF
Successfully loaded HP_cost
Successfully loaded HC_Coverage
Successfully loaded HEF_Main

Dim_Geography (Shape: (52, 4)):
  GEOGRAPHY_KEY Country_Name Region Healthcare_System_Type
0           JPN        Japan  Other              Universal
1           PER         Peru  Other              Universal
2           AUS    Australia  Other              Universal

Dim_Time (Shape: (64, 5)):
   TIME_KEY  Year Quarter  Month FREQ
0      2017  2017      Q4     12    A
1      2019  2019      Q4     12    A
2      2018  2018      Q4     12    A

Dim_InsuranceType (Shape: (7, 5)):
  INSURANCE_TYPE_KEY                                 INSURANCE_TYPE  \
0           TPRIBASI  Public and primary voluntary health insurance   
1           PHINTPHI               Total voluntary health insurance   
2           PHINPMPI             Primary voluntary health insurance   

  Coverage_Level  Deductible_Avg  Copay_Percent  
0       Standard            

In [8]:
import os
import pandas as pd

def read_csvs_as_dfs(directory):
    if not os.path.isdir(directory):
        raise FileNotFoundError(f"Directory not found: {directory}")

    csv_files = sorted([f for f in os.listdir(directory) if f.endswith('.csv')])
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in directory: {directory}")

    dfs = tuple(pd.read_csv(os.path.join(directory, file)) for file in csv_files)
    return dfs




In [ ]:
# Example usage:
df1, df2, df3 = read_csvs_as_dfs(<output_director>)
print(df1.head())
print(df2.head())

[  FINANCING_SCHEME_KEY FINANCING_SCHEME  Revenue_Source
 0                   _T            Total         Premium
 1                   _Z   Not applicable  Direct Payment,
    PROVIDER_KEY                                           PROVIDER  \
 0            _Z                                     Not applicable   
 1            _T                                              Total   
 2           HP3                 Providers of ambulatory healthcare   
 3           HP6                       Providers of preventive care   
 4           HP2              Residential long-term care facilities   
 5           HP1                                          Hospitals   
 6           HP4                    Providers of ancillary services   
 7           HP7  Providers of healthcare system administration ...   
 8           HP8                                Rest of the economy   
 9           HP5     Retailers and other providers of medical goods   
 10           _U                               

In [13]:
df_name_to_df['Dim_FinancingScheme']


,FINANCING_SCHEME_KEY,FINANCING_SCHEME,Revenue_Source
0,_T,Total,Premium
1,_Z,Not applicable,Direct Payment


In [14]:
df_name_to_df['Dim_Provider']


,PROVIDER_KEY,PROVIDER,Provider_Type,Ownership
0,_Z,Not applicable,Clinic,Non-Profit
1,_T,Total,Clinic,Private
2,HP3,Providers of ambulatory healthcare,Pharmacy,Private
3,HP6,Providers of preventive care,Pharmacy,Public
4,HP2,Residential long-term care facilities,Other,Private
5,HP1,Hospitals,Hospital,Non-Profit
6,HP4,Providers of ancillary services,Pharmacy,Private
7,HP7,Providers of healthcare system administration ...,Other,Non-Profit
8,HP8,Rest of the economy,Hospital,Non-Profit
9,HP5,Retailers and other providers of medical goods,Hospital,Public


In [15]:
import os
import pandas as pd

def read_csvs_as_dfs(directory):
    if not os.path.isdir(directory):
        raise FileNotFoundError(f"Directory not found: {directory}")

    csv_files = sorted([f for f in os.listdir(directory) if f.endswith('.csv')])
    if not csv_files:
        raise FileNotFoundError(f"No CSV files found in directory: {directory}")

    dfs = tuple(pd.read_csv(os.path.join(directory, file)) for file in csv_files)
    for i, (df, filename) in enumerate(zip(dfs, csv_files)):
        print(f"DataFrame {i+1} from {filename}:")
        print(f"Columns: {list(df.columns)}\n")
    return dfs

read_csvs_as_dfs("/Users/vidushi/Documents/Datasets/DW/star_schema")


DataFrame 1 from Dim_Demographics.csv:
Columns: ['DEMOGRAPHICS_KEY', 'Age_Group', 'Income_Level', 'Employment_Status']

DataFrame 2 from Dim_FinancingScheme.csv:
Columns: ['FINANCING_SCHEME_KEY', 'FINANCING_SCHEME', 'Revenue_Source']

DataFrame 3 from Dim_Geography.csv:
Columns: ['GEOGRAPHY_KEY', 'Country_Name', 'Region', 'Healthcare_System_Type']

DataFrame 4 from Dim_InsuranceType.csv:
Columns: ['INSURANCE_TYPE_KEY', 'INSURANCE_TYPE', 'Coverage_Level', 'Deductible_Avg', 'Copay_Percent']

DataFrame 5 from Dim_Provider.csv:
Columns: ['PROVIDER_KEY', 'PROVIDER', 'Provider_Type', 'Ownership']

DataFrame 6 from Dim_Time.csv:
Columns: ['TIME_KEY', 'Year', 'Quarter', 'Month', 'FREQ']

DataFrame 7 from Fact_Healthcare_Financing.csv:
Columns: ['Financing_ID', 'TIME_KEY', 'GEOGRAPHY_KEY', 'FINANCING_SCHEME_KEY', 'PROVIDER_KEY', 'OBS_VALUE', 'Per_Capita_Spending', 'Percent_GDP']

DataFrame 8 from Fact_Insurance_Coverage_Outcomes.csv:
Columns: ['Coverage_ID', 'TIME_KEY', 'GEOGRAPHY_KEY', 'INSURA

(   DEMOGRAPHICS_KEY Age_Group Income_Level Employment_Status
 0                 1     18-24          Low           Student
 1                 2     25-34   Medium-Low          Employed
 2                 3     35-44       Medium          Employed
 3                 4     45-54  Medium-High          Employed
 4                 5       55+         High           Retired,
   FINANCING_SCHEME_KEY FINANCING_SCHEME  Revenue_Source
 0                   _T            Total         Premium
 1                   _Z   Not applicable  Direct Payment,
    GEOGRAPHY_KEY                  Country_Name         Region  \
 0            JPN                         Japan          Other   
 1            PER                          Peru          Other   
 2            AUS                     Australia          Other   
 3            CAN                        Canada  North America   
 4            HRV                       Croatia          Other   
 5            KOR                         Korea          Ot